This Python script simulates the dynamics of charged particles under relativistic conditions using the Pygame library. It is designed to visualize and explore concepts such as electromagnetic fields, synchrotron radiation, and the effects of relativity on particle interactions. Here's a breakdown of what the script does:

### Imports and Constants
- **Pygame** is used for graphical rendering and interaction.
- **Numpy** is utilized for numerical operations, especially in vector and matrix computations.
- Constants like `WIDTH`, `HEIGHT`, `SPEED_OF_LIGHT`, and `ELEMENTARY_CHARGE` are defined for use in simulations.

### Synchrotron Function
- `synchrotron_function(x)`: Approximates a function related to synchrotron radiation, which particles emit when they are accelerated in magnetic fields.

### Particle Class
- Represents charged particles with properties like position (`x`, `y`, `z`), `charge`, `mass`, and `velocity`.
- `update`: Updates particle's state based on electromagnetic fields and other particles. It calculates forces from given electric and magnetic fields and updates the particle's velocity and position considering relativistic effects (like mass increase at speeds close to the speed of light and time dilation).
- `synchrotron_radiation_spectrum` and `synchrotron_radiation_power`: Calculate the spectrum and power of synchrotron radiation emitted by the particle.
- `retarded_position` and `retarded_fields`: Compute the position and electromagnetic fields of the particle at previous times, accounting for the finite speed of light (this is part of what makes the simulation 'relativistic').
- `relativistic_mass` and `time_dilation`: Compute the relativistic mass and time dilation effects based on the particle’s velocity.

### ParticleManager Class
- Manages a collection of `Particle` instances.
- `add_particle`: Adds a new particle to the simulation.
- `draw`: Draws all particles on the Pygame screen.
- `update_interactions`: Updates interactions between all particles, considering retarded fields, which simulate the delay in the effect of one particle's field on another due to the finite speed of light.
- `update`: Updates all particles’ states and interactions between them.

### Field Functions
- `electric_field` and `magnetic_field`: Functions to compute the electric and magnetic fields at any point in space based on predefined distributions or configurations.

### Simulation Loop
- Initializes Pygame and sets up the simulation environment.
- Enters a loop where it processes user events (like quitting or adding particles with mouse clicks), updates the state of the simulation (particle movements and interactions), and renders the updated state to the screen.
- Uses Pygame’s drawing functions to visualize particles and possibly field lines or intensity.

### Key Concepts Illustrated by the Script
- **Relativistic Effects**: The script accounts for effects such as time dilation and relativistic mass increase, essential when simulating particles moving at speeds close to the speed of light.
- **Electromagnetic Interactions**: It computes interactions between particles based on electromagnetic fields, including those generated by the particles themselves, considering the retardation effect due to the finite speed of light.
- **Synchrotron Radiation**: It calculates and possibly visualizes the radiation emitted by charged particles when accelerated, which is significant in many high-energy physics contexts.

Overall, this script provides a platform for visualizing and understanding the complex dynamics involved in relativistic particle interactions, including electromagnetic forces and radiation. It serves as an educational tool for illustrating concepts from electromagnetism and relativistic physics.

In [ ]:
# V 3.1 Reletivistic charged particle interaction
#  
# Retarded Electromagnetic _field
# synchrotron_radiation
# fourth-order Runge-Kutta method
# Magnetic field should be lower then 0.1 T

import math
import pygame
import random
import numpy as np
from pygame import surfarray
import colorsys
# Constants
WIDTH, HEIGHT = 800, 600
BLACK = (0, 0, 0)
RED = (255, 0, 0)
ELEMENTARY_MASS = 9.10938356e-31 
SPEED_OF_LIGHT = 299792458  # Speed of light in m/s 
ELEMENTARY_CHARGE = 1.602176634e-19  # Elementary charge in Coulombs
EPSILON_0 = 8.8541878128e-12  # Permittivity of free space in F/m

#particle mass, charge and dt of the simulation. , critical to choose correct values. 
mass = 1e-6
charge = -1
dt = 1e-8

class Particle:
    def __init__(self, x, y, z, charge, mass, dt = 1e-6):
        # Initialize Particle properties
        self.x, self.y, self.z = x, y, z
        self.charge, self.mass = charge, mass

        self.radius = 5  
        self.color = (0, 0, 0)  
        self.velocity = np.array([0.0, 0.0, 0.0])  
        self.acceleration = np.array([0.0, 0.0, 0.0])
        self.dt = dt# 1e-6 #10000 / SPEED_OF_LIGHT
        self.energy_loss_per_second = 0 
        self.total_energy = self.mass

    def update(self, electric_field_function, magnetic_field_function, particles):
        # Calculate base and retarded electric and magnetic fields
        electric_field = electric_field_function(self.x, self.y, self.z) 
        magnetic_field = magnetic_field_function(self.x, self.y, self.z)  
        retarded_electric_field, retarded_magnetic_field = self.calculate_retarded_effects(particles) 
        total_electric_field = electric_field + retarded_electric_field
        total_magnetic_field = magnetic_field + retarded_magnetic_field

        self.update_synchrotron_radiation(total_magnetic_field)
        self.update_color_based_on_radiation()
        
        # Update particle properties based on fields
        force = self.charge * (total_electric_field + np.cross(self.velocity, total_magnetic_field))
        self.acceleration = force / self.mass

        # Update velocity and position using RK4 integration
        self.rk4_step(self.dt, lambda r, v: self.acceleration, lambda r, v: v)

        # Wrap around screen, 
        #self.x %= WIDTH
        #self.y %= HEIGHT                            

    def rk4_step(self, dt, a_func, v_func):
        """Performs a single RK4 integration step for velocity and position."""
        r0 = np.array([self.x, self.y, self.z])
        v0 = self.velocity
        # Combined RK4 calculation step
        def rk4_calc(r, v):
            return v_func(r, v) * dt, a_func(r, v) * dt
        k1 = rk4_calc(r0, v0)
        k2 = rk4_calc(r0 + k1[0]/2, v0 + k1[1]/2)
        k3 = rk4_calc(r0 + k2[0]/2, v0 + k2[1]/2)
        k4 = rk4_calc(r0 + k3[0], v0 + k3[1])
        # Update velocity and position
        self.velocity = v0 + (k1[1] + 2*k2[1] + 2*k3[1] + k4[1]) / 6
        # Speed of light check
        speed = np.linalg.norm(self.velocity)
        if speed > SPEED_OF_LIGHT:
            self.velocity *= 0.999 * SPEED_OF_LIGHT / speed  # More efficient scaling
        self.x, self.y, self.z = r0 + (k1[0] + 2*k2[0] + 2*k3[0] + k4[0]) / 6  
        
    def update_synchrotron_radiation(self, magnetic_field):
        # Calculate the Lorentz factor
        gamma = self.time_dilation()
        # Calculate the unit vector and magnitude of the velocity
        v_mag = np.linalg.norm(self.velocity)
        velocity_unit = self.velocity / (v_mag+1e-31)        
        # Calculate the unit vector of the magnetic field         
        magnetic_field_unit = magnetic_field / np.linalg.norm(magnetic_field)        
        # Calculate the perpendicular component of the velocity relative to the magnetic field
        v_perp = np.cross(velocity_unit, magnetic_field_unit)
        # Calculate the magnitude of the perpendicular velocity
        v_perp_mag = np.linalg.norm(v_perp) * v_mag
        # Calculate the radius of the particle's path due to the magnetic field                
        r = gamma * self.mass * v_perp_mag / (np.abs(self.charge) * np.linalg.norm(magnetic_field))
        # Calculate the acceleration
        if r > 0:
            a_perp = v_perp_mag ** 2 / r
        else:
            a_perp = 0  # Set acceleration to zero if radius is zero
        # Calculate the power radiated due to synchrotron radiation
        P = (2.0 / 3) * (self.charge ** 2 * a_perp ** 2) / (4 * np.pi * EPSILON_0 * SPEED_OF_LIGHT ** 3)
        # Convert power loss to energy loss per time step
        self.energy_loss_per_second = P
        # Update the particle's velocity based on the energy loss
        energy = gamma * self.mass * SPEED_OF_LIGHT ** 2
        new_energy = energy - self.energy_loss_per_second * self.dt
        if new_energy > self.mass * SPEED_OF_LIGHT ** 2:
            new_gamma = new_energy / (self.mass * SPEED_OF_LIGHT ** 2)
            new_v_mag = SPEED_OF_LIGHT * np.sqrt(1 - 1 / (new_gamma ** 2))
            self.velocity = (new_v_mag / v_mag) * self.velocity
        else:
            self.velocity = np.zeros_like(self.velocity)  # Particle stops if energy is depleted

        
    def calculate_retarded_effects(self, particles):
        c = SPEED_OF_LIGHT  # Speed of light in m/s
        epsilon_0 = EPSILON_0  # Vacuum permittivity
        self_position = np.array([self.x, self.y, self.z])
        total_E = np.zeros(3)
        total_B = np.zeros(3)
        for p in particles:
            if p == self:
                continue  # Skip self in the field calculation
            r = self_position - np.array([p.x, p.y, p.z])
            r_mag = np.linalg.norm(r)
            retarded_time = r_mag / c            
            # Assuming constant acceleration for simplicity to calculate retarded position, velocity:
            retarded_position = np.array([p.x, p.y, p.z]) + \
                                            p.velocity * retarded_time + \
                                            0.5 * p.acceleration * retarded_time**2
            retarded_velocity = p.velocity + p.acceleration * retarded_time
            r_retarded = self_position - retarded_position
            r_retarded_mag = np.linalg.norm(r_retarded)
            r_retarded_unit = r_retarded / r_retarded_mag
            # Calculate retarded electric field
            beta = retarded_velocity / c
            gamma = 1 / np.sqrt(1 - np.dot(beta, beta))
            velocity_term = (r_retarded_unit - beta) / (gamma**2 * (1 - np.dot(beta, r_retarded_unit))**3 * r_retarded_mag**2)
            acceleration_term = np.cross(r_retarded_unit, np.cross(r_retarded_unit - beta, p.acceleration / c)) / (c * (1 - np.dot(beta, r_retarded_unit))**3 * r_retarded_mag)
            E = (p.charge / (4 * np.pi * epsilon_0)) * (velocity_term + acceleration_term)
            # Calculate retarded magnetic field
            B = np.cross(r_retarded_unit, E) / c
            total_E += E
            total_B += B

        return total_E, total_B  

    def time_dilation(self):
        # Calculate the time dilation factor        
        speed = np.linalg.norm(self.velocity) 
        #print(speed)
        return 1 / np.sqrt(1 - (speed / SPEED_OF_LIGHT) ** 2)
    
    def relativistic_energy(self):
        # Calculate the relativistic energy of the particle
        gamma = 1 / self.time_dilation()
        return self.mass * SPEED_OF_LIGHT**2 * gamma         
        
    def update_color_based_on_radiation(self):
        # Define the maximum expected power for scaling purposes
        max_power = 3 #SOME_MAX_POWER_VALUE  # You need to define this based on your simulation's parameters
        # Normalize the radiation power to a 0-1 scale
        normalized_power = min(self.energy_loss_per_second / max_power, 1)                 
        # Hue varies from 0.67 (blue) to 0 (red); adjust if you want different start/end colors
        normalized_speed =  np.linalg.norm(self.velocity) / SPEED_OF_LIGHT 
        hue = 0.67 * (1 - normalized_speed  )         
        saturation = 0.5 + 0.5 * normalized_power  # Full saturation
        value = 1        
        # Convert HSV to RGB. colorsys returns floats in the range [0, 1], so convert to [0, 255]
        rgb_float = colorsys.hsv_to_rgb(hue, saturation, value)
        new_color = tuple(int(component * 255) for component in rgb_float)
        self.color =   new_color 
        
class ParticleManager:
    def __init__(self):
        self.particles = []  # List to hold all particles

    def update(self, electric_field, magnetic_field):
        for particle in self.particles:
            particle.update(electric_field, magnetic_field, self.particles)
            
    def add_particle(self, x, y, z, charge, mass,dt):
        # Add a new particle to the list
        self.particles.append(Particle(x, y, z, charge, mass,dt))

    def draw_particles(self, screen):
        """Draws all particles on the screen."""
        for particle in self.particles:
            pygame.draw.circle(screen, particle.color, (int(particle.x), int(particle.y)), particle.radius)

    def draw_bar(self, screen, label, value, max_value, position, color=(0, 255, 0)):
        """Draws a labeled bar for displaying metrics like velocity or time dilation."""
        bar_width = 20
        bar_height = 100
        filled_height = int(bar_height * value / max_value)
        bar_x, bar_y = position
        # Draw the bar outline and filled portion
        pygame.draw.rect(screen, (255, 255, 255), (bar_x, bar_y, bar_width, bar_height), 1)
        pygame.draw.rect(screen, color, (bar_x, bar_y + bar_height - filled_height, bar_width, filled_height))
        # Label the bar
        font = pygame.font.Font(None, 24)
        text = font.render(label, True, (255, 255, 255))
        text_rect = text.get_rect(centerx=bar_x + bar_width // 2, y=bar_y + bar_height + 5)
        screen.blit(text, text_rect)

    def draw(self, screen):
        """Main drawing function."""
        # Constants
        SPEED_OF_LIGHT = 299792458  # meters per second
        MAX_TIME_DILATION = 10  # Arbitrary max for demo purposes
        # Draw all particles
        self.draw_particles(screen)
        # Calculate max metrics
        max_velocity = max(np.linalg.norm(particle.velocity) for particle in self.particles)
        max_time_dilation = max(np.linalg.norm(particle.time_dilation()) for particle in self.particles)
        # Bar positions
        screen_width, screen_height = screen.get_size()
        velocity_bar_position = (screen_width - 60, 160 + (screen_height - 100) // 2)
        time_dilation_bar_position = (velocity_bar_position[0] + 30, velocity_bar_position[1])
        # Draw bars
        self.draw_bar(screen, "V", max_velocity, SPEED_OF_LIGHT, velocity_bar_position)
        self.draw_bar(screen, "T", max_time_dilation, MAX_TIME_DILATION, time_dilation_bar_position)

############################################################################# 
def electric_field_theoretical(x, y, z):
    k = 8.9875517923e9  # Coulomb's constant
    r = np.sqrt(x**2 + y**2 + z**2)
    E_magnitude = k * q / r**2  # Replace q with the source charge
    E_direction = np.array([x, y, z]) / r  # Unit vector in the direction of r
    return E_magnitude * E_direction

def electric_fieldacc(x, y, z):
    # Constants for the accelerator
    center_x, center_y, center_z = 400, 300, 0  # Central point of the accelerator
    epsilon = 1e-9  # Small value to avoid division by zero in arrays
    Q = 1  # Charge creating the field, simplified assumption
    k = 1 / (4 * np.pi * EPSILON_0)  # Coulomb's constant
    # Compute differences from the center and distances
    dx, dy, dz = x - center_x, y - center_y, z - center_z
    distance = np.sqrt(dx**2 + dy**2 + dz**2) + epsilon  # Add epsilon to avoid division by zero
    # Compute radial and angular directions
    radial_direction = np.array([dx, dy, dz]) / distance
    angular_direction = np.array([-dy, dx, np.zeros_like(dz)]) / (np.sqrt(dx**2 + dy**2) + epsilon)  
    # Compute the electric field
    electric_field = Q * k * radial_direction  # Simplified; adjust as needed for your model
    return electric_field

def electric_field_4(x, y, z):
    # Define the electric field for acceleration
    ELECTRIC_FIELD_STRENGTH=10
    ACCELERATO_LENGTH = 10
    if 0 <= z < ACCELERATO_LENGTH:
        return np.array([0, 0, ELECTRIC_FIELD_STRENGTH])  # Constant electric field along the z-axis
    else:
        return np.array([0, 0, 0])  # No electric field outside the accelerator

def magnetic_field_3(x, y, z):
    MAGNETIC_FIELD_STRENGTH=10
    ACCELERATOR_LENGTH=10
    # Define the magnetic field for guiding the particles
    if 0 <= z < ACCELERATOR_LENGTH:
        return np.array([0, MAGNETIC_FIELD_STRENGTH, 0])  # Constant magnetic field along the y-axis
    else:
        return np.array([0, 0, 0])  # No magnetic field outside the accelerator

def electric_field(x, y, z):
    # _chaos  Constants
    #k = 8.9875517873681764e9
    k = 1 / (4 * np.pi * EPSILON_0)
    Q = 1#1e17*ELEMENTARY_CHARGE
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 100, 100, 50  # Standard deviations of the distribution
    Lx, Ly = 800, 600  # Dimensions of the simulation domain
    epsilon = 1e-9  # Small value to avoid division by zero
    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    # Compute the chaotic perturbation
    perturbation = 0.1 * (np.sin(2 * np.pi * 2 * x / Lx - z)
                          + 0.5 * np.cos(3 * np.pi * 6 * y / Ly - 2*z)
                          + 0.2 * np.sin(x*y*z))
    # Combine the Gaussian distribution and chaotic perturbation
    field_magnitude = gaussian #* (1 + 2*perturbation)
    #field_magnitude =   perturbation
    # Compute the electric field components
    field_x = k * field_magnitude * dx / (sigma_x**2 )
    field_y = k * field_magnitude * dy / (sigma_y**2 )
    field_z = k * field_magnitude * dz / (sigma_z**2 )
    return np.array([field_x, field_y, field_z])
         
def magnetic_field(x, y, z):
    # Example: a uniform magnetic field in the z direction
    Bz = 1e-1
    By = 0
    Bx = 0     
    return np.array([Bx, By, Bz ]) # Bx, By, Bz

def electric_field_spiral (x, y, z):
    # Constants
    k = 8.9875517873681764e9
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 50, 50, 50  # Standard deviations of the distribution
    epsilon = 1e-9  # Small value to avoid division by zero
    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    # Compute the radial distance and angle in the xy-plane
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)
    # Compute the spiral factor
    spiral_factor = np.exp(1j * (r / 20))
    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    # Compute the electric field components
    field_x = k * gaussian * (dx / (sigma_x**2 + epsilon) * spiral_factor.real - dy / (sigma_y**2 + epsilon) * spiral_factor.imag)
    field_y = k * gaussian * (dy / (sigma_y**2 + epsilon) * spiral_factor.real + dx / (sigma_x**2 + epsilon) * spiral_factor.imag)
    field_z = k * gaussian * dz / (sigma_z**2 + epsilon)
    return np.array([field_x, field_y, field_z])

#############################################################################

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Particle Simulation")
clock = pygame.time.Clock()

# Particle manager to manage multiple particles
particle_manager = ParticleManager()

particle_manager.add_particle(400, 300, 0, charge, mass*10, dt  )  #
 
x_grid, y_grid, z_grid = np.meshgrid(range(WIDTH), range(HEIGHT), np.array(0))

electric_fields = electric_field(x_grid, y_grid, z_grid)
magnetic_fields = magnetic_field(x_grid, y_grid, z_grid)
field_surface = pygame.Surface((WIDTH, HEIGHT))

normalized_electric_field =electric_fields

ARROW_COLOR = (128, 128, 128)  # Red arrows, for example
ARROW_WIDTH = 1
SCALE_FACTOR = 1e-6  # Adjust this to scale arrow lengths appropriately
GRID_SPACING = 20  # Adjust this to control arrow density 

# Function to draw arrows on the field surface
def draw_arrows(surface, fields, width, height, grid_spacing, scale_factor):
    for x in range(0, width, grid_spacing):
        for y in range(0, height, grid_spacing):
            field_strength = np.linalg.norm(fields[:, y, x])
            field_direction = fields[:, y, x] / (field_strength + 1e-16)  # Normalize and avoid division by zero
            arrow_length = field_strength * scale_factor
            # Convert end position to integer to prevent TypeError
            arrow_end = (
                int(x + arrow_length * field_direction[0]),
                int(y + arrow_length * field_direction[1])
            )
            pygame.draw.line(surface, ARROW_COLOR, (x, y), arrow_end, ARROW_WIDTH)

running = True
while running:
    for event in pygame.event.get():         
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:  # Check for mouse click
            # Get mouse position and add a new particle there
            x, y = pygame.mouse.get_pos()
            #ELEMENTARY_CHARGE# random.choice([-1.0, 1.0])  # Randomly choose charge
            particle_manager.add_particle(x, y,0, charge, mass,dt )  # Add new particle at mouse position          
    
    screen.fill(BLACK) 
    screen.blit(field_surface, (0, 0))     
    draw_arrows(field_surface, normalized_electric_field, WIDTH, HEIGHT, GRID_SPACING, SCALE_FACTOR)

    particle_manager.update(electric_field, magnetic_field)
         
    particle_manager.draw(screen)
    pygame.display.flip()
    clock.tick(60)
pygame.quit()

In [ ]:
#comparing the simulation to the analytical solution of a moving charge partical on a constant magnetic field. 
# Set up the initial conditions
x0, y0, z0 = 0, 0, 0
v0 = 1e6  # Initial velocity magnitude (m/s)
alpha = 0 #np.pi / 4  # Angle between initial velocity and x-y plane (45 degrees)
q = 1 #1.60217663e-19  # Charge of the particle (C)
m = 1e-7 #1e-28 #9.10938356e-31  # Mass of the particle (kg)
dt = 1e-9  # Time step (s)
B = 0.1   # Magnetic field strength (T)
 


# Calculate the cyclotron frequency
omega = (q * B) / m

# Set the initial velocity components
vx0 =  v0 * np.cos(alpha)
vy0 = v0 * np.sin(alpha)
vz0 =0

# Create a particle with the initial conditions
particle = Particle(x0, y0, z0, q, m)
particle.velocity = np.array([vx0, vy0, vz0])
particle.dt= dt

# Define the magnetic field function
def magnetic_field_function(x, y, z):
    return np.array([0, 0, B])

# Define the electric field function
def electric_field_function(x, y, z):
    return np.array([0, 0, 0])

# Simulate the particle motion for a given time
time_steps = 6000
positions = []
positions.append([particle.x, particle.y, particle.z])
for _ in range(time_steps):
    particle.update(electric_field_function, magnetic_field_function, [])
    positions.append([particle.x, particle.y, particle.z])

# Convert positions to numpy array
positions = np.array(positions)

# Calculate analytical positions
t = np.arange(0, time_steps * dt, dt)
x_analytical = (vx0 / omega) * np.sin(omega * t)
y_analytical = (vx0 / omega) * (np.cos(omega * t) - 1)
z_analytical = vz0 * t

# Plot the simulated and analytical trajectories
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], label='Simulated')
ax.plot(x_analytical[0:], y_analytical[0:], z_analytical[0:], label='Analytical')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.show()